In [1]:
import pandas as pd
import pickle
import sqlite3
import numpy as np

In [2]:
opt_RF = pickle.load(open('random_forest.sav', 'rb'))
data = pd.read_csv('output_gdp0_pop0.csv')

In [3]:
if input("load maps? y/n") == 'y':
    gdp = pd.read_csv('gdp_mapped.csv')
    pop = pd.read_csv('pop_mapped.csv')

load maps? y/ny


In [4]:
countries = data.loc[:, 'Area'].unique()

In [5]:
All = slice(None)

pop_c = 'Total population'
cult = 'Cultivated area (arable land + permanent crops)'
gdp_c = 'Gross Domestic Product (GDP)'
trsw = 'Total renewable surface water'
tfw = 'Total freshwater withdrawal (primary and secondary)'
trwr = 'Total renewable water resources'
efr = 'Environmental Flow Requirements'
wsi = 'SDG 6.4.2. Water Stress'

preds = [cult, gdp_c, pop_c, trsw]

trwr_id = 4188
efr_id = 4549

In [6]:
data.set_index(['Area', 'YearBin'], inplace=True)
data.loc[(All, [2020, 2025, 2030]), [gdp_c, pop_c, tfw, wsi]] = None

In [7]:
pop.set_index('mapped', inplace=True)
gdp.set_index('mapped', inplace=True)

In [8]:
def forecast(start, rate, years):
    return start*(1+rate/100)**years

In [9]:
def parm_output(data, gdp_delta, pop_delta):
    new_data = data.copy()

    All = slice(None)
    col_pop = 'Total population'
    for name, group in new_data.groupby('Area'):
        last_val = group.at[(name, 2015), col_pop]
        rate = pop.at[name, 'POPULATION'] + pop_delta
        new_data.at[(name, 2020), col_pop] = forecast(last_val, rate, 5)
        new_data.at[(name, 2025), col_pop] = forecast(last_val, rate, 10)
        new_data.at[(name, 2030), col_pop] = forecast(last_val, rate, 15)

    col_gdp = 'Gross Domestic Product (GDP)'
    world_gdp = 3.2
    for name, group in new_data.groupby('Area'):
        last_val = group.at[(name, 2015), col_gdp]
        if name in gdp.index:
            rate = gdp.at[name, 'gdp_rate'] + gdp_delta
        else:
            rate = world_gdp + gdp_delta
        new_data.at[(name, 2020), col_gdp] = forecast(last_val, rate, 5)
        new_data.at[(name, 2025), col_gdp] = forecast(last_val, rate, 10)
        new_data.at[(name, 2030), col_gdp] = forecast(last_val, rate, 15)

    new_data = new_data.loc[(~new_data.loc[:,pop_c].isna()) &
                    (~new_data.loc[:, cult].isna()),:]

    tfw_preds = opt_RF.predict(new_data.loc[(All, [2020, 2025, 2030]), preds].values)
    new_data.loc[(All, [2020, 2025, 2030]), tfw] = tfw_preds

    new_data[wsi] = 100*new_data[tfw]/(new_data[trwr] - new_data[efr])

    new_data.dropna(axis=0, inplace=True)
    return new_data

In [10]:
final_output = pd.DataFrame(index=data.index)

gdp_parms = [-2, 0, 2]
pop_parms = [-2, 0, 2]
for g in gdp_parms:
    for p in pop_parms:
        new_data = parm_output(data, g, p)
        final_output[f"gdp{g}_pop{p}"] = new_data[wsi]

In [11]:
final_output.loc[(All, [2020, 2025, 2030]), :].to_csv('final_output.csv')
final_output.to_csv('final_output_all_years.csv')